In [1]:
from datasets import load_dataset
from huggingface_hub import from_pretrained_keras
import cv2

import os
os.environ['CURL_CA_BUNDLE'] = ''

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

/home/user/miniconda3/envs/python38/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (None) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/user/miniconda3/envs/python38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-30 14:52:27.093548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 14:52:27.777901: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could 

In [2]:
# tf.config.gpu.set_per_process_memory_growth(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu , True)

2024-05-30 14:52:30.322109: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-30 14:52:30.350233: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-30 14:52:30.350354: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

### Loading the Video Swin Model

In [3]:
swin_model = from_pretrained_keras("Tonic/video-swin-transformer")

/home/user/miniconda3/envs/python38/lib/python3.8/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/user/miniconda3/envs/python38/lib/python3.8/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2024-05-30 14:52:35.390946: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L3

In [4]:
swin_model.summary()

Model: "swin_transformer3d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 projection (PatchEmbed3D)   multiple                  12672     
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 basic_layer (BasicLayer)    multiple                  1799864   
                                                                 
 basic_layer_1 (BasicLayer)  multiple                  3418992   
                                                                 
 basic_layer_2 (BasicLayer)  multiple                  71416800  
                                                                 
 basic_layer_3 (BasicLayer)  multiple                  26757056  
                                                                 
 layer_normalization_51 (La  multiple           

### Frame Analysis

In [5]:
# vid = cv2.VideoCapture("./SSBD/videos/v_ArmFlapping_01.avi")
# ret, frame = vid.read()

# plt.imshow(frame)

frame_lengths = []
for i in range(49):
    cap = cv2.VideoCapture(f"./inputs/clipped/armflapping/{i}.avi")
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # print("Number of frames : " , length)
    frame_lengths.append(length)
    
frame_lengths = pd.DataFrame(np.array(frame_lengths))
frame_lengths.describe()
# sorted(frame_lengths)

,0
count,49.000000
mean,255.081633
std,338.717547
min,1.000000
25%,70.000000
50%,156.000000
75%,306.000000
max,1656.000000


### Testing the video swin transformer

In [6]:
idx = 3 #video number

frames = []
video = cv2.VideoCapture(f"./inputs/clipped/armflapping/{idx}.avi")
while True:
    read, frame= video.read()
    if not read:
        break
    frames.append(frame)
frames = np.array(frames)

frames.shape

(456, 224, 224, 3)

In [7]:
CLIP_SIZE = 32
# BATCH_SIZE = 1
frame_size = frames.shape[0]
num_batches = (frame_size // 32)
clipped_frames_size = num_batches * 32
clipped_frames = frames[:clipped_frames_size]
clipped_frames = clipped_frames.reshape(num_batches , 32 , 224 , 224 , 3)
clipped_frames = np.rollaxis(clipped_frames, 4, 1)
# # clipped_frames = clipped_frames[np.newaxis , ...]
# clipped_frames = clipped_frames.astype(np.float32)
clipped_frames = clipped_frames.astype(np.uint8)
clipped_frames = np.average(clipped_frames , axis=0)
print("Video Averaged every 32 frames : ", clipped_frames.shape)

Video Averaged every 32 frames :  (3, 32, 224, 224)


In [8]:
outp = swin_model.predict(clipped_frames[np.newaxis , ...])
# outp = swin_model(clipped_frames)
outp.shape , type(outp)

2024-05-30 14:53:13.376567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902


1/1 [==============================] - 3s 3s/step


((1, 1024, 16, 7, 7), numpy.ndarray)

In [9]:
outp.max() , outp.min()

(5.9015255, -5.5870867)

In [10]:
NUM_CLASSES = 3

def build_model():
    inp = tf.keras.layers.Input(shape = (3,32,224,224))
    x = swin_model(inp)
    x = tf.keras.layers.GlobalAveragePooling3D(data_format="channels_first")(x)
    # x = tf.keras.layers.Flatten(data_format="channels_first")(x)
    x = tf.keras.layers.Dense(units = 64 , activation='relu' , input_shape=(1024,))(x)
    x = tf.keras.layers.Dense(units = NUM_CLASSES , activation='softmax')(x)
    
    model = tf.keras.Model(inputs = [inp] , outputs = x)
    return model

modelx = build_model()
opt = tf.keras.optimizers.AdamW(3e-5 , 1e-2)
# opt = Adam(3e-5)
modelx.compile(optimizer= opt, loss="categorical_crossentropy", metrics=['accuracy'])
modelx.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 32, 224, 224   0         
                             )]                                  
                                                                 
 swin_transformer3d (SwinTr  (None, 1024, 16, 7, 7)    103407432 
 ansformer3D)                                                    
                                                                 
 global_average_pooling3d (  (None, 1024)              0         
 GlobalAveragePooling3D)                                         
                                                                 
 dense (Dense)               (None, 64)                65600     
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                             

### Creating Dataset

In [11]:
DATA = [
    # DIRECTORY , CLASS
    ("./inputs/clipped/armflapping" , 0),
    ("./inputs/clipped/headbanging" , 1),
    ("./inputs/clipped/spinning" , 2)
]

DATA_32 = [] #contains videos more than 32 frames

for data in DATA:
    vids = [f for f in os.listdir(data[0]) if os.path.isfile(os.path.join(data[0], f))]
    for vid in vids:
        cap_dir = f"{data[0]}/{vid}"
        cap = cv2.VideoCapture(cap_dir)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if length >= 32:
            DATA_32.append((cap_dir , data[1])) #directory of video and class of video
            
df = pd.DataFrame(DATA_32 , columns=['video' , 'label'])
df.head()

,video,label
0,./inputs/clipped/armflapping/28.avi,0
1,./inputs/clipped/armflapping/34.avi,0
2,./inputs/clipped/armflapping/20.avi,0
3,./inputs/clipped/armflapping/2.avi,0
4,./inputs/clipped/armflapping/17.avi,0


In [12]:
class Dataset_Generator(tf.keras.utils.Sequence) :
    def __init__(self, videos, labels, batch_size) :
        self.videos = videos
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self) :
        return int(len(self.videos) / self.batch_size)

    def __getitem__(self, idx) :
        batch_x = self.videos[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

        
        vids = []
        labels = []
        for vid , label in zip(batch_x , batch_y):
            video = cv2.VideoCapture(vid)
            frames = []
            while True:
                read, frame= video.read()
                if not read:
                    break
                frames.append(frame)
            frames = np.array(frames)
            frames = frames[:(len(frames) // 32) * 32]
            
            frame_size = frames.shape[0]
            num_batches = (frame_size // 32)
            clipped_frames_size = num_batches * 32
            clipped_frames = frames[:clipped_frames_size]
            clipped_frames = clipped_frames.reshape(num_batches , 32 , 224 , 224 , 3)
            clipped_frames = np.rollaxis(clipped_frames, 4, 1)
            clipped_frames = clipped_frames.astype(np.float32)
            clipped_frames = np.average(clipped_frames , axis=0)
            vids.append(clipped_frames)
            # print(clipped_frames.shape)
            
            l = [0 for i in range(NUM_CLASSES)]
            l[label] = 1
            labels.append(l)

        return np.array(vids).astype(np.uint8) , np.array(labels)

In [13]:
BATCH_SIZE = 1

validation_split = 0.3
# validation_split_len = int(validation_split * SAMPLES)

# df_sampled = df.sample(frac=1).reset_index(drop=True) #shuffle the dataset

# df_train = df_sampled[: -validation_split_len]
# df_test = df_sampled[-validation_split_len : - validation_split_len // 2]
# df_val = df_sampled[- validation_split_len // 2 : ]

X = df["video"]
Y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=validation_split, random_state=42)

validation_test_split = len(X_test) // 2

training_gen = Dataset_Generator(
    X_train.to_list(), 
    y_train.to_list(),
    BATCH_SIZE
)

validation_gen = Dataset_Generator(
    X_test[:validation_test_split].to_list(), 
    y_test[:validation_test_split].to_list(),
    BATCH_SIZE
)

test_gen = Dataset_Generator(
    X_test[validation_test_split:].to_list(), 
    y_test[validation_test_split:].to_list(),
    BATCH_SIZE
)

len(X_train) , len(X_test)

(65, 28)

In [14]:
x , y = training_gen.__getitem__(0)
x.shape

(1, 3, 32, 224, 224)

In [15]:
from sys import getsizeof

print("Size of one batch : " , getsizeof(x) // 1024 // 1024 , "MBs")

Size of one batch :  4 MBs


### Training the Model

In [16]:
# modelx(clipped_frames)

modelx.fit(training_gen , epochs=5 , validation_data=validation_gen)

Epoch 1/5


2024-05-28 11:35:26.514747: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 150.06MiB (rounded to 157351936)requested by op model_1/swin_transformer3d/StatefulPartitionedCall/StatefulPartitionedCall/basic_layer_2/swin_transformer_block3d_17/window_attention3d_17/Softmax
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-05-28 11:35:26.514804: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-05-28 11:35:26.514812: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 304, Chunks in use: 302. 76.0KiB allocated for chunks. 75.5KiB in use in bin. 2.5KiB client-requested in use in bin.
2024-05-28 11:35:26.514816: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 65, Chunks in use: 65. 34.2K

ResourceExhaustedError: Graph execution error:

Detected at node 'basic_layer_2/swin_transformer_block3d_17/window_attention3d_17/Softmax' defined at (most recent call last):
Node: 'basic_layer_2/swin_transformer_block3d_17/window_attention3d_17/Softmax'
OOM when allocating tensor with shape[4,16,784,784] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node basic_layer_2/swin_transformer_block3d_17/window_attention3d_17/Softmax}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_176841]